# 显示支气管气道树3D模型

In [1]:
import SimpleITK as sitk
import numpy as np

def load_CT_scan_3D_image(niigz_file_name):
    itkimage = sitk.ReadImage(niigz_file_name)
    numpyImages = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImages, numpyOrigin, numpySpacing


def save_CT_scan_3D_image(image, origin, spacing, niigz_file_name):
    if type(origin) != tuple:
        if type(origin) == list:
            origin = tuple(reversed(origin))
        else:
            origin = tuple(reversed(origin.tolist()))
    if type(spacing) != tuple:
        if type(spacing) == list:
            spacing = tuple(reversed(spacing))
        else:
            spacing = tuple(reversed(spacing.tolist()))

    itkimage = sitk.GetImageFromArray(image, isVector=False)
    itkimage.SetSpacing(spacing)
    itkimage.SetOrigin(origin)
    sitk.WriteImage(itkimage, niigz_file_name, True)

def compute_fusioned_airway_tree_3d_model(label_npy, predict_npy):
    assert label_npy.shape == predict_npy.shape
    
    depth, height, width = label_npy.shape
    airway_tree_3d_model = np.zeros((depth, height, width))
    
    false_positive_count = 0
    false_negative_count = 0
    true_positive_count = 0
    true_negative_count = 0
    
    for depth_index in range(depth):
        for height_index in range(height):
            for width_index in range(width):
                gt_voxel = label_npy[depth_index, height_index, width_index]
                pred_voxel = predict_npy[depth_index, height_index, width_index]
                
                if pred_voxel == 1 and gt_voxel == 0:  # False Positive
                    false_positive_count += 1
                    airway_tree_3d_model[depth_index, height_index, width_index] = 2
                elif pred_voxel == 1 and gt_voxel == 1:  # True Positive
                    true_positive_count += 1
                    airway_tree_3d_model[depth_index, height_index, width_index] = 3
                elif pred_voxel == 0 and gt_voxel == 0:  # True Negative
                    true_negative_count += 1
                    airway_tree_3d_model[depth_index, height_index, width_index] = 0
                elif pred_voxel == 0 and gt_voxel == 1:  # False Negative
                    false_negative_count += 1
                    airway_tree_3d_model[depth_index, height_index, width_index] = 1
    
    return airway_tree_3d_model, [false_positive_count, 
                                  false_negative_count, 
                                  true_positive_count, 
                                  true_negative_count]
    

In [2]:
ATM_054_groundtruth_file_path = "./ATM_054_0000-groundtruth.nii.gz"
ATM_054_predict_file_path = "./ATM_054_0000-predict.nii.gz"
ATM_054_raw_image_file_path = "./ATM_054_0000_clean_hu.nii.gz"

gt_npy, origin, spacing = load_CT_scan_3D_image(ATM_054_groundtruth_file_path)
predict_npy, predict_origin, predict_spacing = load_CT_scan_3D_image(ATM_054_predict_file_path)
raw_image, raw_origin, raw_spacing = load_CT_scan_3D_image(ATM_054_raw_image_file_path)

In [3]:
print(gt_npy.shape, origin, spacing)

(528, 247, 341) [ 455.23001099  -84.69999695 -207.3999939 ] [0.5        0.83007812 0.83007812]


In [4]:
assert gt_npy.shape == predict_npy.shape == raw_image.shape
assert origin.all() == predict_origin.all() == raw_origin.all()
assert spacing.all() == predict_spacing.all() == raw_spacing.all()

In [5]:
airway_tree, stats = compute_fusioned_airway_tree_3d_model(gt_npy, predict_npy)

print(stats)

# 0         black                             blackground
# 1         red                               False Negative
# 2         green                             False Positive
# 3         blue                              True Positive
save_CT_scan_3D_image(airway_tree, origin, spacing, "ATM_054_0000_airway_tree_with_3colors.nii.gz")

[16139, 7571, 196520, 44251626]


In [6]:
airway_tree_with_2colors = gt_npy + predict_npy

# 0          black                              blackground
# 1          red                                False Positive + False Negative
# 2          green                              True Positive
save_CT_scan_3D_image(airway_tree_with_2colors, origin, spacing, 
                      "ATM_054_0000_airway_tree_with_2colors.nii.gz")

In [7]:
assert sum(stats) == (gt_npy.shape[0] * gt_npy.shape[1] * gt_npy.shape[2])
print(sum(stats))


44471856


# Calculation for the airway tree metrics

In [8]:
import numpy as np

def dice_coefficient_score_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    intersection = np.sum(pred * label)
    dice_coefficient_score = round(((2.0 * intersection + smooth) / (np.sum(pred) + np.sum(label) + smooth)) * 100, 2)
    return dice_coefficient_score

def false_positive_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fp = np.sum(pred - pred * label) + smooth
    fpr = round(fp * 100 / (np.sum((1.0 - label)) + smooth), 3)
    return fpr

def false_negative_rate_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    fn = np.sum(label - pred * label) + smooth
    fnr = round(fn * 100 / (np.sum(label) + smooth), 3)
    return fnr

def sensitivity_calculation(pred, label):
    sensitivity = round(100 - false_negative_rate_calculation(pred, label), 3)
    return sensitivity

def specificity_calculation(pred, label):
    specificity = round(100 - false_positive_rate_calculation(pred, label), 3)
    return specificity

def precision_calculation(pred, label, smooth=1e-5):
    pred = pred.flatten()
    label = label.flatten()
    tp = np.sum(pred * label) + smooth
    precision = round(tp * 100 / (np.sum(pred) + smooth), 3)
    return precision

def specificity_calculation(pred, label):
    specificity = round(100 - false_positive_rate_calculation(pred, label), 3)
    return specificity

def tree_length_calculation(pred, label_skeleton, smooth=1e-5):
    pred = pred.flatten()
    label_skeleton = label_skeleton.flatten()
    tree_length = round((np.sum(pred * label_skeleton) + smooth) / (np.sum(label_skeleton) + smooth) * 100, 2)
    return tree_length

def branch_detected_calculation(pred, label_parsing, label_skeleton, thresh=0.8):
    label_branch = label_skeleton * label_parsing
    label_branch_flat = label_branch.flatten()
    label_branch_bincount = np.bincount(label_branch_flat)[1:]
    total_branch_num = label_branch_bincount.shape[0]
    pred_branch = label_branch * pred
    pred_branch_flat = pred_branch.flatten()
    pred_branch_bincount = np.bincount(pred_branch_flat)[1:]
    if total_branch_num != pred_branch_bincount.shape[0]:
        lack_num = total_branch_num - pred_branch_bincount.shape[0]
        pred_branch_bincount = np.concatenate((pred_branch_bincount, np.zeros(lack_num)))
    branch_ratio_array = pred_branch_bincount / label_branch_bincount
    branch_ratio_array = np.where(branch_ratio_array >= thresh, 1, 0)
    detected_branch_num = np.count_nonzero(branch_ratio_array)
    detected_branch_ratio = round((detected_branch_num * 100) / total_branch_num, 2)
    return total_branch_num, detected_branch_num, detected_branch_ratio

In [9]:
false_positive_voxel, false_negative_voxel, true_positive_voxel, true_negative_voxel = stats

print(false_positive_voxel, false_negative_voxel, true_positive_voxel, true_negative_voxel)

16139 7571 196520 44251626


In [10]:
False_Positive_Rate = false_positive_voxel / (false_positive_voxel + true_negative_voxel)
print(False_Positive_Rate * 100)

print("FPR = ", false_positive_rate_calculation(predict_npy, gt_npy))

0.03645767975862346
FPR =  0.036


In [11]:
False_Negative_Rate = false_negative_voxel / (false_negative_voxel + true_positive_voxel)
print(False_Negative_Rate * 100)

print("FNR = ", false_negative_rate_calculation(predict_npy, gt_npy))

3.7096197284544643
FNR =  3.71


In [89]:
True_Positive_Rate = true_positive_voxel / (true_positive_voxel + false_negative_voxel)
print(True_Positive_Rate * 100)

print("TPR = ", sensitivity_calculation(predict_npy, gt_npy))

96.29038027154554
TPR =  96.29


In [12]:
DSC = (2 * true_positive_voxel) / (true_positive_voxel + false_positive_voxel + true_positive_voxel + false_negative_voxel)
print(DSC * 100)

print("DSC = ", dice_coefficient_score_calculation(predict_npy, gt_npy))

94.31073785242951
DSC =  94.31


In [18]:
Precision = true_positive_voxel / (false_positive_voxel + true_positive_voxel)
print(Precision * 100)

print("Precision = ", precision_calculation(predict_npy, gt_npy))

92.41085493677672
Precision =  92.411
